In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor
import seaborn as sns
sns.set_style("dark")
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

#from warnings import filterwarnings
#filterwarnings('ignore')

data=None
def absolute_correlations(col, df=data):
    #absolute_values = np.abs(df[col])
    corrs = pd.DataFrame(df.select_dtypes(include=[np.number]).corrwith(df[col]), columns=['correlation'])
    corrs['absol'] = np.abs(corrs['correlation'])
    return corrs.sort_values('absol', ascending=False).drop('absol', axis=1).tail(len(corrs)-1)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [57]:
data = pd.read_csv('telco_churn.csv')
del data['customerID']

data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.65,Yes


In [58]:
for i in data.columns:
    if 'Yes' in data[i].values:
        data[i] = np.where(data[i] == 'Yes', 1, 0)

data.dtypes

C:\Users\Vince\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  from ipykernel import kernelapp as app


gender               object
SeniorCitizen         int64
Partner               int32
Dependents            int32
tenure                int64
PhoneService          int32
MultipleLines         int32
InternetService      object
OnlineSecurity        int32
OnlineBackup          int32
DeviceProtection      int32
TechSupport           int32
StreamingTV           int32
StreamingMovies       int32
Contract             object
PaperlessBilling      int32
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int32
dtype: object

In [59]:
data['TotalCharges'] = np.where(
    data['TotalCharges'] == ' ',
    np.nan,
    data['TotalCharges']
)

data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)
data['TotalCharges'] = data['TotalCharges'].astype(float)

In [60]:
internet = pd.get_dummies(data['InternetService'], prefix='internet')
del data['InternetService']
data = pd.concat([data, internet], axis=1)

contract = pd.get_dummies(data['Contract'], prefix='contract')
del data['Contract']
data = pd.concat([data, contract], axis=1)

payment = pd.get_dummies(data['PaymentMethod'], prefix='PaymentMethod')
del data['PaymentMethod']
data = pd.concat([data, payment], axis=1)

gender = pd.get_dummies(data['gender'], prefix='gender')
del data['gender']
data = pd.concat([data, gender], axis=1)

In [61]:
data.dtypes

SeniorCitizen                                int64
Partner                                      int32
Dependents                                   int32
tenure                                       int64
PhoneService                                 int32
MultipleLines                                int32
OnlineSecurity                               int32
OnlineBackup                                 int32
DeviceProtection                             int32
TechSupport                                  int32
StreamingTV                                  int32
StreamingMovies                              int32
PaperlessBilling                             int32
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int32
internet_DSL                                 uint8
internet_Fiber optic                         uint8
internet_No                                  uint8
contract_Month-to-month        

In [63]:
x = data.drop('Churn', axis=1)
y = data['Churn']